![jupyter](.\Week1-03.png)

1.sigmoid输出在（0,1）之间，通过设置阈值可以完成二分类

In [1]:
#导入库
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

### 1 生成数据

In [7]:
sample = 100
mean = 1
bias = 1
n_data = torch.ones(sample, 2)
x0 = torch.normal(mean * n_data) + bias
y0 = torch.zeros(sample)
x1 = torch.normal(-mean * n_data) + bias
y1 = torch.ones(sample)
train_x = torch.cat((x0, x1), 0)
train_y = torch.cat((y0, y1), 0)

### 2 选择模型

In [9]:
class LR(nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.features = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.features(x)
        x = self.sigmoid(x)
        return x

lr_net = LR()

### 3,4 损失函数和优化器

In [5]:
loss_fn = nn.BCELoss()
lr = 0.01
optimizer = torch.optim.SGD(lr_net.parameters(),lr=lr,momentum=0.9)

### 5.模型训练

In [8]:
for iteration in range(1000):
    
    #forward
    y_pred = lr_net(train_x)
    
    #caculate loss
    loss = loss_fn(y_pred.squeeze(),train_y)
    
    #backward
    loss.backward()
    
    #update para
    optimizer.step()
    
    #clean gradient
    optimizer.zero_grad()
    
    #plot
    if iteration % 20 == 0:
        mask = y_pred.ge(0.5).float().squeeze()
        correct = (mask==train_y).sum()
        acc = correct.item()/ train_y.size(0)
        
        plt.scatter(x0.data.numpy()[:, 0], x0.data.numpy()[:, 1], c='r', label='class 0')
        plt.scatter(x1.data.numpy()[:, 0], x1.data.numpy()[:, 1], c='b', label='class 1')
        
        w0, w1 = lr_net.features.weight[0]
        w0, w1 = float(w0.item()), float(w1.item())
        plot_b = float(lr_net.features.bias[0].item())
        plot_x = np.arange(-6, 6, 0.1)
        plot_y = (-w0 * plot_x - plot_b) / w1
        
        plt.xlim(-5, 7)
        plt.ylim(-7, 7)
        plt.plot(plot_x, plot_y)

        plt.text(-5, 5, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size': 20, 'color': 'red'})
        plt.title("Iteration: {}\nw0:{:.2f} w1:{:.2f} b: {:.2f} accuracy:{:.2%}".format(iteration, w0, w1, plot_b, acc))
        plt.legend()

        plt.show()
        plt.pause(0.5)
        
        if acc > 0.99:
            break

TypeError: __init__() takes 1 positional argument but 2 were given